## Test the MIRI imager distortion solution (standalone and JWST pipeline versions)  ##

### Set basic environment stuff ###

In [1]:
import os as os
import numpy as np
from numpy.testing import assert_allclose
import pdb as pdb

Import the MIRI coordinates code from https://github.com/STScI-MIRI/miricoord and ensure that it is on the PYTHONPATH.  Also ensure that the local data directory environmental variable is set:

setenv MIRICOORD_DATA_DIR /YourLocalPathToData/ (this is where io will happen)

In [2]:
data_dir=os.path.expandvars('$MIRICOORD_DATA_DIR')

### First we'll test the standalone distortion solution (mirim_tools) ###

In [3]:
import miricoord.imager.mirim_tools as mt
mt.set_toolversion('flt1')

Define a point on the imager (0-indexed convention) and convert to v2,v3 assuming F770W filter

In [4]:
x,y=692.5,511.5

In [5]:
v2,v3=mt.xytov2v3(x,y,'F770W')

In [6]:
print(v2,v3)

[-453.37849012] [-373.8105493]


The V2,V3 location of the imager reference point should be -453.378, -373.811 in FLT-1

Convert back to x,y pixel coordinates

In [7]:
xback,yback=mt.v2v3toxy(v2,v3,'F770W')

In [8]:
print(xback,yback)

[692.50070198] [511.49949074]


The returned pixel coordinates should be the same as the original pixel coordinates to within a small fraction of a pixel (roundtrip error)

Now run a full set of tests using both F770W and F1800W filters.  If no errors, there were no problems:

In [9]:
mt.testtransform()

In [10]:
### Test with mirim_pipetools ###

In [11]:
import miricoord.imager.mirim_pipetools as mpt

In [12]:
x,y=692.5,511.5

In [13]:
v2,v3=mpt.xytov2v3(x,y,'F770W')

In [14]:
print(v2,v3)

-453.37849011784346 -373.8105493033865


The V2,V3 location of the imager reference point should be -453.378, -373.811 in FLT-1

Convert back to x,y pixel coordinates

In [15]:
xback,yback=mpt.v2v3toxy(v2,v3,'F770W')

In [16]:
print(xback,yback)

692.5007019828922 511.4994907442945


The returned pixel coordinates should be the same as the original pixel coordinates to within a small fraction of a pixel (roundtrip error)

Now run a full set of tests using both F770W and F1800W filters.  If no errors, there were no problems:

In [17]:
mpt.testtransform()

### Now we'll test the implementation of the distortion solution in the JWST pipeline (mirim_pipetools) ###

In [18]:
from astropy.modeling import models
from asdf import AsdfFile
from jwst import datamodels
from jwst.assign_wcs import miri

Point to a F770W imager observation from APT 1029 (MIR-013)

In [19]:
file=data_dir+'testing/jw01029001001_04101_00001_mirimage_flt1_cal.fits'

Read the datamodel from the file and print some information about it to see the available reference frames.

In [20]:
im = datamodels.ImageModel(file)

In [21]:
print(im.meta.wcs)

   From      Transform   
---------- --------------
  detector  CompoundModel
      v2v3 DVA_Correction
v2v3vacorr       v23tosky
     world           None


In [22]:
print(im.meta.wcs.available_frames)

['detector', 'v2v3', 'v2v3vacorr', 'world']


Define a point at the reference point of the imager and check that the v2,v3 is correct

In [23]:
x,y=692.5,511.5

In [24]:
v2,v3=im.meta.wcs.transform("detector", "v2v3", x,y)

In [25]:
print(v2,v3)

-453.37849011784346 -373.8105493033865


The V2,V3 location of the imager reference point should be -453.378, -373.811 in FLT-1

Check that RA/DEC look reasonable

In [26]:
ra, dec = im.meta.wcs(x, y)

In [27]:
print(ra,dec)

82.44925834604759 -69.9662691183138


Do a quick test transform with all filters

In [28]:
x,y=692.5,511.5
v2,v3=mt.xytov2v3(x,y,'F560W')
v2,v3=mt.xytov2v3(x,y,'F770W')
v2,v3=mt.xytov2v3(x,y,'F1000W')
v2,v3=mt.xytov2v3(x,y,'F1130W')
v2,v3=mt.xytov2v3(x,y,'F1280W')
v2,v3=mt.xytov2v3(x,y,'F1500W')
v2,v3=mt.xytov2v3(x,y,'F1800W')
v2,v3=mt.xytov2v3(x,y,'F2100W')
v2,v3=mt.xytov2v3(x,y,'F2550W')
v2,v3=mt.xytov2v3(x,y,'F2550WR')
v2,v3=mt.xytov2v3(x,y,'FND')
v2,v3=mt.xytov2v3(x,y,'F1065C')
v2,v3=mt.xytov2v3(x,y,'F1140C')
v2,v3=mt.xytov2v3(x,y,'F1550C')
v2,v3=mt.xytov2v3(x,y,'F2300C')

Test the bounding boxes (all of the following should be nan)

In [29]:
im.meta.wcs(1.49,500)

(82.38970167581016, -69.97196561136897)

In [30]:
im.meta.wcs(1027.6,500)

(82.47768426365609, -69.96292623305237)

In [31]:
im.meta.wcs(500,-0.6)

(nan, nan)

In [32]:
im.meta.wcs(500,1023.6)

(nan, nan)

In [33]:
im.meta.wcs.bounding_box

ModelBoundingBox(
    intervals={
        x0: Interval(lower=-0.5, upper=1031.5)
        x1: Interval(lower=-0.5, upper=1023.5)
    }
    model=CompoundModel(inputs=('x0', 'x1'))
    order='C'
)